In [1]:
#This file aims to preprocess the provided dataset into our NEQsi score classification cateogries
import pandas as pd

In [2]:
#load in silver training set and gold test set (replace file path to your file path)
df_silver = pd.read_csv("/Users/pranavsomani/Downloads/Blastocyst_Dataset/Gardner_train_silver.csv", sep=';')
df_gold = pd.read_csv("/Users/pranavsomani/Downloads/Blastocyst_Dataset/Gardner_test_gold_onlyGardnerScores.csv", sep=';')

In [3]:
#add one to both EXP categories to match label process in NEQsi paper
df_silver['EXP_silver'] = df_silver['EXP_silver'] + 1
df_gold['EXP_gold'] = df_gold['EXP_gold'] + 1

In [4]:
#replace NA values with 1 as told to do so in NEQsi paper
df_gold["EXP_gold"].fillna(1, inplace = True)
df_gold.fillna("ND", inplace=True) 

In [5]:
#translate numerical scores into their alphabetical counterparts. This information is then later used to calculate NEQsi scores
def ICM_filter(num):
    if num == 0:
        return "A"
    elif num == 1:
        return "B"
    elif num == 2:
        return "C"
    elif num == 3:
        return "ND"
    elif type(num) == None:
        return "ND"
def ICM_filter_gold(num):
    if num == '0':
        return "A"
    elif num == '1':
        return "B"
    elif num == '2':
        return "C"
    elif num == 'ND':
        return "ND"
def TE_filter(num):
    if num == 0:
        return "A"
    elif num == 1:
        return "B"
    elif num == 2:
        return "C"
    elif num == 3:
        return "D"
def TE_filter_gold(num):
    if num == '0':
        return "A"
    elif num == '1':
        return "B"
    elif num == '2':
        return "C"
    elif num == 'ND':
        return "ND"
df_silver['ICM_silver'] = df_silver['ICM_silver'].apply(lambda x: ICM_filter(x))
df_gold['ICM_gold'] = df_gold['ICM_gold'].apply(lambda x: ICM_filter_gold(x))
df_silver['TE_silver'] = df_silver['TE_silver'].apply(lambda x: TE_filter(x))
df_gold['TE_gold'] = df_gold['TE_gold'].apply(lambda x: TE_filter_gold(x))

In [7]:
#combine alphabetical scores to match NEQsi calculation
df_silver["Combined"] = df_silver["ICM_silver"] + df_silver["TE_silver"]
df_gold["Combined"] = df_gold["ICM_gold"] + df_gold["TE_gold"]

In [9]:
#return numerical value corresponding to alphabetical classifications
def label_func(val):
    if val == "AA":
        return 5
    elif (val == "AB" or val == "BA"):
        return 4
    elif(val == "AC" or val == "BB" or val == "CA"):
        return 3
    elif(val == "CB" or val == "BC"):
        return 2
    elif(val == "CC"):
        return 1
    else:
        return 0

df_silver["Combined_values"] = df_silver["Combined"].apply(lambda x: label_func(x))
df_gold["Combined_values"] = df_gold["Combined"].apply(lambda x: label_func(x))

In [10]:
df_gold['EXP_gold'] = df_gold['EXP_gold'].replace({'ND': 1})

In [11]:
#return NEQsi scores for each blastocyst
df_silver["NESQI_scores"] = df_silver["EXP_silver"] + df_silver["Combined_values"]
df_gold["NESQI_scores"] = df_gold["EXP_gold"] + df_gold["Combined_values"]

In [12]:
#categorize each NEQsi score as its corresponding bucket/class
def nesqi_bin_label(num):
    if num <= 3:
        return "Poor"
    elif (num >=4 and num <= 7):
        return "Fair"
    elif (num >= 8):
        return "Great"

df_silver["label"] = df_silver["NESQI_scores"].apply(lambda x: nesqi_bin_label(x))
df_gold["label"] = df_gold["NESQI_scores"].apply(lambda x: nesqi_bin_label(x))

In [13]:
#save labeled dataset
df_silver.to_csv("silver_labeled.csv")
df_gold.to_csv("gold_labeled.csv")